In [1]:
import torch

from safetensors.torch import load_file
from esm.models.esm3 import ESM3
from transformers import AutoModel
from esm.sdk.api import ESMProtein, GenerationConfig

/home/av47/miniconda3/envs/esm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_model = ESM3.from_pretrained("esm3-open", device=torch.device("cuda:1"))
new_model = ESM3.from_pretrained("esm3-open", device=torch.device("cuda:1"))

Fetching 22 files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 65863.45it/s]
/home/av47/miniconda3/envs/esm/lib/python3.12/site-packages/esm/pretrained.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any

In [3]:
def load_weights(path, model):
    state_dict = torch.load(path, map_location="cuda:1")
    new_dict = {}
    
    for k, v in state_dict.items():
        if k in new_model.state_dict():
            new_dict[k] = v
    model.load_state_dict(new_dict)

In [4]:
load_weights("weights/20241130-180831/checkpoint-12657/pytorch_model.bin", new_model)

/tmp/ipykernel_3764681/3923622363.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location="cuda:1")


In [12]:
prompt = "EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLEWVARIYPTNGYTRYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYC__________WGQGTLVTVSS"
protein = ESMProtein(sequence=prompt)
base_protein = base_model.generate(protein, GenerationConfig(track="sequence", num_steps=4, temperature=0.1))
new_protein = new_model.generate(protein, GenerationConfig(track="sequence", num_steps=4, temperature=0.1))
print(base_protein.sequence[96:106], new_protein.sequence[96:106])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 17.37it/s]

ARGYGYYFDY ARWDGGYFDY
